✅ THE PLAN — MVP API with Current Building Blocks
We’ll do this in 4 main parts:

🔹 1. Core Models
Node

Relationship

🔹 2. Ontology & Validation
Ontology rules

Validator for relationships

🔹 3. Semantic Classifier
Classifies incoming sentences (into types like “Geographic” → located_in)

🔹 4. Pipeline Logic
Sentence ➝ Semantic label ➝ Relationship ➝ Node Graph

All validated before being stored



In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 100.9 MB/s eta 0:

In [ ]:
!pip install pydantic


In [ ]:
import chromadb
from flask import Flask, request, jsonify
import pandas as pd
import json
from datetime import datetime


In [ ]:
# Initialize Flask app
app = Flask(__name__)

In [ ]:
# Initialize ChromaDB client
client = chromadb.Client()
xml_file_path = r'/content/Aalborg_ChromaDB_Entry.xml'


In [ ]:

df = pd.read_xml(xml_file_path)
# Create a collection (or retrieve if already exists)
collection = client.create_collection(name="node_graphs_collection")

# Sample DataFrame (you already have this `df_grouped_nodes` with 'node_graph' column)
# need to debug this....Going to notebooks to debug and check structure of it in the database-
display(df)

,index,title,parent_node,child_node,sentence_restructured,original_fragment,node_graph,meta_namespace,timestamp,meta_topic,semantic_tags
0,0,'Aalborg',"Aalborg, Aarhus, situation, Odense, Limfjord","city seaport, Denmark seat, bishop town, count...",Aalborg is a city.Aalborg is a seaport of Denm...,'Aalborg is a city''seaport of Denmark''the se...,"{'content': ""' Aalborg is a city and seaport o...",Geographical Entities,2003-07-14 09:23:44+00:00,"'Municipalities in Denmark', 'Geographical Loc...","'Aalborg Municipality', 'City', 'Administrativ..."
1,1,'Aalborg_Architecture',"Aalborg, foundation, Church, bishopric, Ålborg...","houses, century, the 17th century Aalborg, cen...",Aalborg has picturesque houses dating back to ...,'An old castle''some picturesque houses of the...,"{'content': ""'An old castle and some picturesq...",Geographical Entities,2003-07-14 09:23:44+00:00,"'Municipalities in Denmark', 'Geographical Loc...","'Aalborg Municipality', 'City', 'Administrativ..."
2,2,'Aalborg_Industry',Aalborg,"climate, Aarhus center grain center, industry ...",Aalborg has a climate similar to Aarhus.Aalbor...,'Aalborg is a growing industrial''commercial c...,"{'content': ""'Aalborg is a growing industrial ...",Geographical Entities,2003-07-14 09:23:44+00:00,"'Municipalities in Denmark', 'Geographical Loc...","'Aalborg Municipality', 'City', 'Administrativ..."
3,3,'Aalborg_Surroundings',"Aalborg, Vildmose, Southeast, Limfjord, Nørres...","lake, west northwest, Aalborg Vildmose moor, D...",Limfjord broadens into an irregular lake to th...,'The situation is typical of the north of Jutl...,"{'content': ""'The situation is typical of the ...",Geographical Entities,2003-07-14 09:23:44+00:00,"'Municipalities in Denmark', 'Geographical Loc...","'Aalborg Municipality', 'City', 'Administrativ..."


In [ ]:
display(df.iloc[0]['node_graph'])
#

'{\'content\': "\' Aalborg is a city and seaport of Denmark, the seat of a bishop, and chief town of the county of North Jutland, on the south bank of the Limfjord, which connects the North Sea and the Kattegat. Aalborg is the 4th largest city in Denmark after Copenhagen, Aarhus and Odense\'", \'parent_child_structure\': {\'Aalborg\': {\'Functional\': [\'city\', \'seaport\', \'Denmark\'], \'Geographic\': [\'seat\', \'bishop\', \'town\', \'county\', \'Jutland\', \'bank\', \'Limfjord\', \'Kattegat\']}, \'Limfjord\': {\'Geographic\': [\'Sea\']}, \'Aarhus\': {\'Geographic\': [\'city\', \'Denmark\', \'Aalborg\']}, \'Odense\': {\'Geographic\': [\'city\', \'Denmark\', \'Aalborg\']}, \'situation\': {\'Geographic\': [\'Aalborg\', \'north\', \'Jutland\']}}}'

In [ ]:
display(df.iloc[1]['node_graph'])

'{\'content\': "\'An old castle and some picturesque houses of the 17th century remain. The cathedral Budolphi church dates mostly from the middle of the 18th century, while the Vor Frue church was partially burnt in 1894, but the foundation of both is of the 14th century or earlier. There are also an ancient hospital and a museum of art and antiquities. On the north side of the Limfjord is Nørresundby, connected with Aalborg by a road bridge, by an iron railway bridge as well as by a motorwaytunnel under the Limfjord. Aalborg received town privileges in 1342 and the bishopric dates from 1554. Note that both spellings forms Aalborg and Ålborg are used in Danish language. ==External links== http://www.visitaalborg.com/ Aalborg Tourist office http://www.auc.dk/ Aalborg University\'", \'parent_child_structure\': {\'Aalborg\': {\'Geographic\': [\'houses\', \'century\', \'the 17th century\', \'sector\'], \'Functional\': [\'hospital\', \'museum\', \'antiquities\', \'houses\', \'century\', \'

In [ ]:
f_grouped_nodes = pd.DataFrame(data)

# Helper function to insert node graphs into ChromaDB
def insert_node_graphs_into_chromadb(df):
    for index, row in df.iterrows():
        node_graph_data = {
            "content": row['node_graph']['content'],
            "parent_child_structure": json.dumps(row['node_graph']['parent_child_structure']),
            "meta_namespace": 'Geographical Entities',
            "timestamp": datetime.utcnow().isoformat(),
            "meta_topic": 'Municipalities in Denmark, Geographical Locations, Cities in North Jutland',
            "semantic_tags": 'Aalborg Municipality, City, Administrative Division, Political Entity, Region'
        }

        # Insert into ChromaDB
        collection.add(
            documents=[node_graph_data["content"]],
            metadatas=[{
                "parent_child_structure": node_graph_data["parent_child_structure"],
                "meta_namespace": node_graph_data["meta_namespace"],
                "timestamp": node_graph_data["timestamp"],
                "meta_topic": node_graph_data["meta_topic"],
                "semantic_tags": node_graph_data["semantic_tags"]
            }],
            ids=[str(index)]
        )


In [ ]:
@app.route('/insert_node_graphs', methods=['POST'])
def insert_node_graphs():
    # Insert the node_graph data into ChromaDB
    insert_node_graphs_into_chromadb(df_grouped_nodes)
    return jsonify({"message": "Node graphs inserted successfully!"}), 200

# Helper function to retrieve node graph by ID
@app.route('/retrieve_node_graph/<string:node_id>', methods=['GET'])
def retrieve_node_graph(node_id):
    # Query ChromaDB for the node graph using the ID
    result = collection.query(query_texts=[node_id], n_results=1)

    if result['documents']:
        retrieved_data = result['documents'][0]
        # Parse the parent-child structure
        parent_child_structure = json.loads(result['metadatas'][0]["parent_child_structure"])
        return jsonify({
            "content": retrieved_data,
            "parent_child_structure": parent_child_structure
        }), 200
    else:
        return jsonify({"message": "Node graph not found!"}), 404

# Route for querying node graphs based on semantic tags (example)
@app.route('/query_node_graphs', methods=['GET'])
def query_node_graphs():
    tags = request.args.get('tags', '').split(',')

    # Search for documents with the provided tags
    result = collection.query(query_texts=tags, n_results=5)

    if result['documents']:
        return jsonify({
            "documents": result['documents'],
            "metadatas": result['metadatas']
        }), 200
    else:
        return jsonify({"message": "No matching node graphs found!"}), 404

if __name__ == '__main__':
    app.run(debug=True)

✅ Output I am aiming to Get
* Clean semantic mapping from sentence

* Validated ontology relationships

* Rejection on invalid logic

* Nodes and relationships saved in memory



In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
import uuid

class Node(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    label: str
    type: str
    content: Optional[str] = None
    tags: Optional[List[str]] = []

class Relationship(BaseModel):
    source: str
    target: str
    type: str


I had to think about this. Humanity only has maybe one or two strong persuits of ontology. Example an English Major would be strong in contextual and grammactial understanding. An English Major doesn't always hold strong ontological properties in Math or other subjects. So when I thought about this, I had to choose a basic moral compass with restrictions to protect an AI from bad influences in the API, but also choose the like what "Major" I could bring strong ontology into the AI. Like choosing a college major that clicks with a strong relatable capacity within the AI. Geography and Medicine, Environment seems a good start, with small ontological relationships in other area's.


In [ ]:
# ontology library....I had to think about this...
# this will be moved over to yaml
# I am just using notebook for debugging and scribble

ontology_registry = {
    "environment": {
        "affects": {
            "allowed_source_types": ["Pollution", "Climate Zone", "Human Activity", "Environmental Threat"],
            "allowed_target_types": ["Ecosystem", "Species", "Habitat", "Biome"],
            "description": "Describes how environmental factors impact natural systems.",
            "inference_rules": [
                "If A affects B AND B is_a Ecosystem THEN A indirectly affects all Species in B"
            ]
        },
        "found_in": {
            "allowed_source_types": ["Species", "Natural Resource", "Pollution"],
            "allowed_target_types": ["Ecosystem", "Habitat", "Biome"],
            "description": "Indicates natural or unnatural presence within an area.",
            "inference_rules": []
        },
        "is_a": {
            "allowed_source_types": ["Species", "Climate Zone", "Pollution", "Ecosystem", "Natural Resource"],
            "allowed_target_types": ["Environmental Entity"],
            "description": "Defines categorization in environmental taxonomy.",
            "inference_rules": [
                "If A is_a B AND B is_a C THEN A is_a C"
            ]
        },
        "caused_by": {
            "allowed_source_types": ["Pollution", "Deforestation", "Environmental Threat"],
            "allowed_target_types": ["Human Activity"],
            "description": "Links harmful outcomes to human activities.",
            "inference_rules": []
        },
        "regulated_by": {
            "allowed_source_types": ["Environmental Threat", "Human Activity"],
            "allowed_target_types": ["Government", "NGO", "Policy"],
            "description": "Shows control or mitigation relationship.",
            "inference_rules": []
        },
        "preserves": {
            "allowed_source_types": ["Human Activity", "NGO", "Policy"],
            "allowed_target_types": ["Species", "Habitat", "Ecosystem"],
            "description": "Positive action taken to sustain environment.",
            "inference_rules": []
        }
    },

    "geography": {
        "located_in": {
            "allowed_source_types": ["City", "Region", "Municipality", "Island", "Mountain", "River"],
            "allowed_target_types": ["Region", "Country", "Continent", "Ocean"],
            "description": "Entity physically or politically located within a broader area.",
            "inference_rules": [
                "If A located_in B AND B part_of C THEN A located_in C"
            ]
        },
        "part_of": {
            "allowed_source_types": ["Municipality", "Region", "Island", "Province"],
            "allowed_target_types": ["Region", "Country", "Continent"],
            "description": "Defines structural inclusion within a larger political or natural entity.",
            "inference_rules": []
        },
        "is_a": {
            "allowed_source_types": ["City", "Region", "Country", "Island", "Mountain", "River", "Lake"],
            "allowed_target_types": ["Geographical Entity", "Administrative Division", "Natural Feature"],
            "description": "Defines a type-subtype relationship.",
            "inference_rules": [
                "If A is_a B AND B is_a C THEN A is_a C"
            ]
        },
        "borders": {
            "allowed_source_types": ["Country", "Region", "State"],
            "allowed_target_types": ["Country", "Region", "State"],
            "description": "Indicates adjacency or geopolitical border.",
            "inference_rules": [
                "If A borders B THEN B borders A"
            ]
        },
        "capital_of": {
            "allowed_source_types": ["City"],
            "allowed_target_types": ["Country", "Region", "State"],
            "description": "Denotes official administrative capital.",
            "inference_rules": [
                "If A capital_of B THEN A is_a City AND B is_a Political Entity"
            ]
        }
    },

    "medical": {
        "treats": {
            "allowed_source_types": ["Drug", "Procedure", "Therapy"],
            "allowed_target_types": ["Disease", "Condition", "Symptom"],
            "description": "The treatment or substance intended to cure or alleviate a condition.",
            "inference_rules": [
                "If A treats B AND B has_symptom C THEN A may alleviate C"
            ]
        },
        "has_symptom": {
            "allowed_source_types": ["Disease", "Condition"],
            "allowed_target_types": ["Symptom"],
            "description": "Defines a symptom commonly associated with a disease.",
            "inference_rules": []
        },
        "caused_by": {
            "allowed_source_types": ["Disease", "Condition", "Symptom"],
            "allowed_target_types": ["Virus", "Bacteria", "Genetic Mutation", "Environmental Factor"],
            "description": "Identifies the source or cause of a medical issue.",
            "inference_rules": [
                "If A caused_by B THEN B causes A"
            ]
        },
        "diagnosed_by": {
            "allowed_source_types": ["Disease", "Condition"],
            "allowed_target_types": ["Test", "Scan", "Procedure"],
            "description": "Indicates diagnostic method for a condition.",
            "inference_rules": []
        },
        "affects": {
            "allowed_source_types": ["Disease", "Condition"],
            "allowed_target_types": ["Organ", "Tissue", "Body System"],
            "description": "Defines the physiological target of a disease or condition.",
            "inference_rules": []
        },
        "interacts_with": {
            "allowed_source_types": ["Drug"],
            "allowed_target_types": ["Drug"],
            "description": "Describes pharmacological interaction, often with contraindications.",
            "inference_rules": [
                "If A interacts_with B THEN B interacts_with A"
            ]
        },
        "is_a": {
            "allowed_source_types": ["Disease", "Symptom", "Organ", "Drug"],
            "allowed_target_types": ["Medical Entity", "Biological Structure", "Chemical Compound"],
            "description": "Type-subtype categorization in medical taxonomy.",
            "inference_rules": [
                "If A is_a B AND B is_a C THEN A is_a C"
            ]
        }
    }

    # TODO: Add more shallow or deep domains (e.g., emotional, historical, legal)
}



In [ ]:
def classify_relationship_type(sentence: str) -> str:
    sentence = sentence.lower()

    if any(kw in sentence for kw in ['located', 'near', 'by', 'on', 'at', 'north', 'south', 'east', 'west']):
        return "Geographic"
    elif any(kw in sentence for kw in ['larger', 'smaller', 'compared', 'better']):
        return "Comparison"
    elif any(kw in sentence for kw in ['is', 'has', 'includes', 'offers']):
        return "Functional"
    else:
        return "Unknown"

def semantic_to_ontology_type(semantic_label: str) -> Optional[str]:
    return {
        "Geographic": "located_in",
        "Comparison": "compared_to",  # You can add later
        "Functional": "is_a"
    }.get(semantic_label, None)
